REQUIREMENTS.TXT
EXAMPLE_MD_TO_TEXT.TXT
HF_TOKEN.txt

In [1]:
with open("example_md_to_text.txt", "r", encoding="latin-1") as f: # TO DO: check proper encoding for .md files
    markdown_example = f.read()

with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

In [2]:
!pip install -r requirements.txt

  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached asttokens-3.0.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached attrs-24.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached datasets-3.2.0-py3-none-any.whl.metadata (20 kB)
  Using cached debugpy-1.8.12-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
  Using cached decorator-5.1.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached executing-2.1.0-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached filelock-3.16.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cach

In [3]:
! python -m spacy download fr_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 39.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
from pprint import pprint
import tqdm
# pprint(markdown_example)

Basic OLlama Workflow

In [5]:
# import ollama

In [6]:
# # basic workflow of getting llama embeddings with ollama
# ollama.embeddings(model="llama3.2:3b",
#                 prompt="Hello")

In [7]:
# # basic workflow of generating responses with ollama
# ollama.generate(model="llama3.2:3b",
#                 prompt="Whos is Obama")

Loading HF to get Attention Masks

In [8]:
with open("HF_TOKEN.txt", "r") as f:
    hf_token = f.read()

In [9]:
from huggingface_hub import HfFolder, whoami

HfFolder.save_token(hf_token)
print(whoami()["name"])

alberto-lorente


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
import torch

device = "cpu"
if torch.cuda.is_available():
  print("Cuda available")
  device = "cuda:0"

Cuda available


In [11]:
# using huggingface tokenizer for attettion layer
# make sure you have GPU enabled
# takes around 5 mins to load with TPUs

from transformers import AutoTokenizer, AutoModel

model_id = "HIT-TMG/KaLM-embedding-multilingual-mini-v1" # which model to use?
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_id)
model = AutoModel.from_pretrained(model_id).to(device)

In [12]:
model

Qwen2Model(
  (embed_tokens): Embedding(151936, 896)
  (layers): ModuleList(
    (0-23): 24 x Qwen2DecoderLayer(
      (self_attn): Qwen2Attention(
        (q_proj): Linear(in_features=896, out_features=896, bias=True)
        (k_proj): Linear(in_features=896, out_features=128, bias=True)
        (v_proj): Linear(in_features=896, out_features=128, bias=True)
        (o_proj): Linear(in_features=896, out_features=896, bias=False)
      )
      (mlp): Qwen2MLP(
        (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
        (up_proj): Linear(in_features=896, out_features=4864, bias=False)
        (down_proj): Linear(in_features=4864, out_features=896, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
    )
  )
  (norm): Qwen2RMSNorm((896,), eps=1e-06)
  (rotary_emb): Qwen2RotaryEmbedding()
)

- text enriching with VLLM to add a little description of tables before hey appear?

- embeddings that return attention layer

- perform similarity on the attention layer -> cluster similar sentences

- summarize the clusters

- tree-based graph

## Splitting the text into sentences and paragraphs

In [13]:
import spacy

nlp = spacy.load("fr_core_news_sm")

In [14]:
doc = nlp(markdown_example)
sents = [sent.text for sent in doc.sents]

In [15]:
sents[0:10]

['# Séance publique\n\n\n\n# du jeudi 28 octobre 2021\n\n\n\n# à 18h00\n\n\n\n# Chorum',
 'Alain Gilles',
 '- Halle Vacheresse\n\n\n\n#',
 'Rue des Vernes à Roanne\n\n\n\n#',
 'PROCES',
 'VERBAL\n\n\n\n',
 "L'an deux mille vingt et un, le 28 octobre à 18 h 00, les conseillers communautaires de Roannais Agglomération, se sont réunis à l\x92Espace Chorum \x96 Halle",
 'Vacheresse \x96',
 'Rue des Vernes à Roanne.\n\n\n\n',
 'La convocation de tous les conseillers a été faite le 22 octobre 2021, dans les formes et délais prescrits par la loi, par Yves Nicolin, Président.\n\n\n\n']

In [16]:
import numpy as np

# to reconstruct based on sentences, we can iterate through the list with a range and
# concat every 10 sents into a single string

rang_sentence_union = np.arange(start=0, stop=len(sents), step=10)
# print(rang)
print("Total number of sents: ", len(sents))
print("Number of final chunks: ", len(rang_sentence_union))


Total number of sents:  913
Number of final chunks:  92


In [17]:
paragraphs = []
i = 0

# merging sentences based on the ranges
while i+1 < len(rang_sentence_union):
  start = rang_sentence_union[i]
  stop = rang_sentence_union[i+1]
  # print(start, stop)

  subset_to_join = sents[start : stop]
  sent_union = " ".join(subset_to_join)

  paragraph_info = {"paragraph_union": sent_union,
                    "start_range": start,
                    "stop_range": stop,
                    "list_sents":subset_to_join}

  paragraphs.append(paragraph_info)

  i += 1

# if the stop of the range comes before the last sentence, we take those final couple of sentences
# and add them to the last sentence of the paragraph list

if stop != len(sents):

  subset_to_join = sents[stop : len(sents)]
  final_sents = " ".join(subset_to_join)
  para_to_edit = paragraphs.pop(-1)
  final_union = para_to_edit["paragraph_union"] + " " + final_sents

  para_to_edit["paragraph_union"] =  final_union
  para_to_edit["stop_range"] = len(sents)
  para_to_edit["list_sents"].extend(subset_to_join)

  paragraphs.append(para_to_edit)

In [18]:
paragraphs[-1]

{'paragraph_union': '# Dotations aux amortissements et aux provisions : 47 000 \x80\n\n\n\nAjustement provision pour dépréciation des actifs circulants (-26 k\x80) et provision pour Compte Epargne Temps (73 k\x80).\n\n\n\n# Virement à la section d\x92investissement : 118 500 \x80\n\n\n\n(autofinancement complémentaire)\n\n\n\n# Recettes :\n\n\n\n# Produits des services : -257 140 \x80\n\n\n\n Il s\x92agit principalement de la diminution des recettes du Nauticum en raison du COVID et de la mauvaise météo de cet été (-300 k\x80) et des billetteries Pleine nature (-42,14 k\x80) et culturelles (-41,47 k\x80), d\x92un complément pour les revenus des coupes de bois (59,07 k\x80) et à l\x92ajustement de refacturations (67,4 k\x80).\n\n\n\n # Impôts et taxes : 3 600 \x80\n\n\n\n Ajustement du reversement des taxes foncières par la commune de Mably sur les bâtiments situés dans la ZAC de Mably. \n\n\n\n # Subventions - Dotations : -45 060 \x80\n\n\n\nAjustement',
 'start_range': 900,
 'stop_ran

Now we have a list of dictionaries, one for each constructed paragraph. So far, each paragraph has the union string, start and end indexes as well as the list of individual sentences that were joined.

## Getting the text embeddings


NOTE: There is one important caveat. We are computing sentence embeddings. If we were to pass the whole doc, the attention mask for each token would look very different.

### Example for one piece of text

In [19]:
# dir(tokenizer)

In [20]:
tokenized_markdown = tokenizer(markdown_example, return_tensors="pt").to(device)

In [21]:
len(tokenized_markdown[0])

20468

In [22]:
model.eval()
with torch.no_grad():
  # first batch (only one since we are processing one doc)
  # final token
    embeddings = model(**tokenized_markdown)[0][:, 0].squeeze(0)

In [23]:
embeddings.shape

torch.Size([896])

We take the last token since it's the one which contains all the aggregate info?

In [24]:
# embeddings

In [25]:
import torch.nn.functional as F

normalized_embeddings = F.normalize(embeddings, p=2, dim=0)

In [26]:
# normalized_embeddings

### For a list of sentences

In [27]:
def compute_embeddings(sentence):

    tokenized_sentences = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True).to(device)

    model.eval()
    with torch.no_grad():
        embeddings = model(**tokenized_sentences)[0][:, 0].squeeze(0) # to take out the unused dimension since we are not batching
        # print(embeddings.shape)

    normalized_embeddings = F.normalize(embeddings, p=2, dim=0)
    detached_embeddings = normalized_embeddings.detach().cpu().numpy() # detached into cpu so that we can manipulate them for clustering

    torch.cuda.empty_cache() # careful with running out of memory

    return detached_embeddings

In [28]:
list_paras = [para["paragraph_union"] for para in paragraphs]

In [29]:
for paras, para_dict in zip(list_paras, paragraphs):
    para_dict["para_embedding"] = compute_embeddings(paras)

In [30]:
paragraphs[0]["para_embedding"].shape

(896,)

Now there is an extra key for the embeddings that we will use for the clsutering.

## Clustering

In [31]:
unsqueezeded_embeddings = [para_dict["para_embedding"] for para_dict in paragraphs]

In [32]:
# unsqueezeded_embeddings[0:2]

### Example for one number of clusters

In [33]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [34]:
gm = GaussianMixture(n_components=4, random_state=42)
clusters = gm.fit_predict(unsqueezeded_embeddings)

In [35]:
# clusters # label assigned to each sentence

In [36]:
sil_sc = silhouette_score(unsqueezeded_embeddings, clusters)
print(sil_sc) # the closer to 1 the better (how similar is an object to its cluster compared to the other clusters)

0.5186369


In [37]:
clusters

array([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 0, 1, 3, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0,
       0, 0, 3, 0, 3, 3, 3, 3, 1, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

### Optimal number of clusters

In [38]:
import numpy as np

# initial number of clusters to try
range_clusters = np.arange(start=3, stop=9, step=1)
print(range_clusters)

[3 4 5 6 7 8]


We could compute jensenshannon distance as well.

In [39]:
def cluster_n(n_clusters, embeddings):

  gm = GaussianMixture(n_components=n_clusters, random_state=42)
  clusters = gm.fit_predict(embeddings)
  sil_sc = silhouette_score(unsqueezeded_embeddings, clusters)

  print("Number of clusters: ", n_clusters)
  print("Score: ", sil_sc)
  print()

  return clusters, sil_sc

In [40]:
silhouette_scores = []
clusters_labels = []
for n_cluster in range_clusters:
  clusters, sil_sc = cluster_n(n_cluster, unsqueezeded_embeddings)
  silhouette_scores.append(sil_sc)
  clusters_labels.append(clusters)

Number of clusters:  3
Score:  0.5307558

Number of clusters:  4
Score:  0.5186369

Number of clusters:  5
Score:  0.35501358

Number of clusters:  6
Score:  0.35773662

Number of clusters:  7
Score:  0.3963203

Number of clusters:  8
Score:  0.47093514



In [41]:
max = np.argmax(silhouette_scores)
optimal_n = range_clusters[max]
print("Index", max)
print("Optimal Number of Clusters", optimal_n)

Index 0
Optimal Number of Clusters 3


In [42]:
final_clusters = clusters_labels[max]

## Logging Information, concatinating the text relating to each cluster and recomputing embeddings

In [63]:
clusters_ids = {f"cluster_{cluster_id}": {"para_indexes": [],
                                          "union_paras": ""} for cluster_id in np.arange(0, optimal_n, 1)}
clusters_ids

{'cluster_0': {'para_indexes': [], 'union_paras': ''},
 'cluster_1': {'para_indexes': [], 'union_paras': ''},
 'cluster_2': {'para_indexes': [], 'union_paras': ''}}

In [64]:
# saving the cluster in the paragraphs dictionary
# and the indexes of the paragraphs of cluster in another one

i = 0

for para_dict, cluster in zip(paragraphs, final_clusters):
  cluster_n_string = f"cluster_{cluster}"
  para_dict["para_cluster"] = cluster_n_string
  clusters_ids[cluster_n_string]["para_indexes"].append(i)
  clusters_ids[cluster_n_string]["union_paras"] = clusters_ids[cluster_n_string]["union_paras"] + para_dict["paragraph_union"]
  i = i + 1

In [69]:
# depending on the post-processing techniques, we may not be using these embeddings
# and we'll have to re-tokenize and embed with the model we use

for cluster in clusters_ids.keys():
  text = clusters_ids[cluster]["union_paras"]
  cluster_embds = compute_embeddings(text)
  clusters_ids[cluster]["cluster_embedding"] = cluster_embds

In [66]:
# clusters_ids["cluster_0"]
# looking at this example, we can see that the clustering is mostly sequential

In [70]:
paragraphs[0].keys()

dict_keys(['paragraph_union', 'start_range', 'stop_range', 'list_sents', 'para_embedding', 'para_cluster'])

In [71]:
clusters_ids.keys()

dict_keys(['cluster_0', 'cluster_1', 'cluster_2'])

In [72]:
clusters_ids["cluster_0"].keys()

dict_keys(['para_indexes', 'union_paras', 'cluster_embedding'])

RECAP

So far we have:
- the reconstructed text based on the paragraph clusters.
- the embeddings of the reconstructed clusters.
- the reconstructed paragraphs in ranges of 10 sentences.
- the embeddings of the paragraphs detached and squeezed.
- the cluster each paragraph belongs to.
- the indexes of the paragraphs of each cluster.
- the actual sentences split from SpaCy, in case we need them after.


LEFT TO DO FOR THE PREPROCESS: GET THE NUMBER/TABLES

## Sumarization

Now that we have the basic separation wrt to clusters, we can start creating different representations for each cluster and doing the hierarchical indexing.

In [ ]:
from transformers import AutoModelForSeq2SeqLM

summ_model = 'csebuetnlp/mT5_multilingual_XLSum' # take the one for mt5 french!!

tokenizer_summ = AutoTokenizer.from_pretrained(summ_model)
model_summ = AutoModelForSeq2SeqLM.from_pretrained(summ_model).to(device)

In [ ]:
import re
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip())) # was part of the docs

### Individual example

In [ ]:
example_cluster = " ".join(cluster_aggregates[5]["texts"])
example_cluster

In [ ]:
tokenized_example = tokenizer_summ(
                              [WHITESPACE_HANDLER(example_cluster)],
                              return_tensors="pt",
                              padding="max_length",
                              truncation=True
                          ).to(device)

In [ ]:
tokenized_example

In [ ]:
input_ids = tokenized_example["input_ids"]
# input_ids = tokenized_example["input_ids"].squeeze(dim=0) NOP,expects the unsqueexed dimension

In [ ]:
input_ids

In [ ]:
len(input_ids[0])

In [ ]:
output = model_summ.generate(
    input_ids=input_ids,
    max_length=1000,
    no_repeat_ngram_size=2,
    num_beams=4) # if it is too slow, adjust num_beams and max_len

In [ ]:
output[0]

In [ ]:
summary = tokenizer_summ.decode(
    output[0],
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

In [ ]:
summary

### For the clusters

In [ ]:
from torch.amp import autocast

In [ ]:
def generate_summary(concat_text):

  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

  tokenized_example = tokenizer_summ(
                              [WHITESPACE_HANDLER(concat_text)],
                              return_tensors="pt",
                              padding="max_length",
                              truncation=True).to(device)

  input_ids = tokenized_example["input_ids"]

  model_summ.eval()
  with autocast("cuda"):
    summary_embedding_output = model_summ.generate(
                                              input_ids=input_ids,
                                              max_length=1000,
                                              no_repeat_ngram_size=2,
                                              num_beams=4) # if it is too slow, adjust num_beams and max_len

  summary_decoded = tokenizer_summ.decode(
                                    summary_embedding_output[0],
                                    skip_special_tokens=True,
                                    clean_up_tokenization_spaces=False)

  torch.cuda.empty_cache() # Careful with running out of memory

  return summary_embedding_output.detach().cpu().numpy(), summary_decoded

In [ ]:
torch.cuda.empty_cache()

In [ ]:
def process_cluster(cluster, cluster_aggregates=cluster_aggregates):

  print("Procesing cluster ", cluster+1)
  torch.cuda.empty_cache()

  cluster_aggregates[cluster]["broader_representations"] = {}

  concat_text = " ".join(cluster_aggregates[cluster]["texts"])
  cluster_aggregates[cluster]["broader_representations"]["concat_text"] = concat_text


  cluster_embeddings = compute_embeddings(concat_text)
  cluster_aggregates[cluster]["broader_representations"]["cluster_sentence_embeddings"] = cluster_embeddings[0] # taking out the extra unnecesary dimension

  torch.cuda.empty_cache()

  summary_embedding_output, summary_decoded = generate_summary(concat_text)
  cluster_aggregates[cluster]["broader_representations"]["cluster_summary_embedding"] = summary_embedding_output
  cluster_aggregates[cluster]["broader_representations"]["cluster_summary_decoded"] = summary_decoded

  return cluster_aggregates


In [ ]:
for cluster in list(cluster_aggregates.keys()):
  cluster_aggregates = process_cluster(cluster)

In [ ]:
cluster_aggregates[1]["broader_representations"].keys()

In [ ]:
cluster_aggregates[1]["broader_representations"]["cluster_summary_decoded"]

In [ ]:
pprint(cluster_aggregates[1]["broader_representations"]["concat_text"])

In [ ]:
cluster_aggregates[0]["broader_representations"]["cluster_summary_decoded"]